In [ ]:
from urllib.request import Request, urlopen
import json
import csv
import time
import logging
from datetime import datetime
import os

In [ ]:
def getChatFormatted(infos):
  chat = []
  for comment in infos['comments']:
    message = {'seg': comment['content_offset_seconds']}
    message['body'] = comment['message']['body']
    if ('user_badges' in comment['message']):
      ids = ''
      for badges in comment['message']['user_badges']:
        if(ids == ''):
          ids = badges['_id']
        else:
          ids += ','+badges['_id']
      message['user_badges'] = ids

    chat.append(message)
  
  return chat
  
  

In [ ]:
def writeFile(fileName, chat):
  keys = ['seg', 'body', 'user_badges']

  if not(os.path.isfile('./'+fileName)):
      with open(fileName, 'a', newline='', encoding="utf-8") as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(chat)
  else:
    with open(fileName, 'a', newline='', encoding="utf-8") as output_file:
      dict_writer = csv.DictWriter(output_file, keys)
      dict_writer.writerows(chat)

In [ ]:
def getChatInfo(url):
  try:
    req = Request(url)
    req.add_header('client-id', os.getenv('CLIENT_ID'))
    content = urlopen(req).read().decode('UTF-8')
    infos = json.loads(content)
    return infos

  except req.Timeout as err:
    logging.error({"message": err.message})

In [ ]:
def getChat(url):
  infos = getChatInfo(url)
  chat = getChatFormatted(infos)

  writeFile(fileName, chat)

  if('_next' in infos):
    time.sleep(3)
    nextChat(infos['_next'])


def nextChat(nextUrl):
  url = 'https://api.twitch.tv/v5/videos/'+videoId+'/comments?cursor='+nextUrl
  getChat(url)
    
def getChatbySec(sec):
  url = 'https://api.twitch.tv/v5/videos/'+videoId+'/comments?content_offset_seconds='+sec
  getChat(url)

videoId = '1314356920'
now = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
fileName = 'chat_'+videoId+'_'+now+'.csv'

getChatbySec('0')